# **Objectives and Hypotheses**

## **Primary Hypotheses:**
### Season-wise Hypothesis
Can season-specific models better capture trends in bike demand than a unified model?

## **Experimental Design:**
### Baseline Model:
XGBoost Regressor
### Target Variable:
Rented Bike Count
### Dataset:
Seoul Bike Sharing Demand
### Data Split:
80% training, 20% testing

### Visualization:
PCA for dimensionality reduction and cluster visualization

### Performance Metrics:
RMSE, MAE, and R²

## **Expected Outcomes:**


# **Import Libraries**

In [ ]:
pip install ucimlrepo

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.decomposition import PCA
import math
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.spatial.distance import mahalanobis
from ucimlrepo import fetch_ucirepo
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.stats.outliers_influence import variance_inflation_factor


from matplotlib.colors import LinearSegmentedColormap
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D


plt.style.use('seaborn-v0_8-whitegrid')
mpl.rcParams['axes.edgecolor'] = '#333333'
mpl.rcParams['axes.linewidth'] = 0.8
mpl.rcParams['xtick.color'] = '#333333'
mpl.rcParams['ytick.color'] = '#333333'


# **Import Data**

In [ ]:
# Fetch Seoul Bike Sharing Demand dataset from UCI ML Repository
seoul_bike_sharing_demand = fetch_ucirepo(id=560)

In [ ]:
# Data (as pandas dataframes)
X_original = seoul_bike_sharing_demand.data.features
y_original = seoul_bike_sharing_demand.data.targets


In [ ]:
# Print dataset information
print("Dataset Metadata:")
print(seoul_bike_sharing_demand.metadata)
print("\nVariable Information:")
print(seoul_bike_sharing_demand.variables)

In [ ]:
# Examine feature information
print("\nOriginal feature columns:")
print(X_original.columns.tolist())
print("\nOriginal target variable:")
print(y_original.columns.tolist())

# **Data Preprocessing**

In [ ]:
# Make 'Rented Bike Count' the new target if it exists

if 'Rented Bike Count' in X_original.columns:
    # Make 'Rented Bike Count' the new target
    y = X_original[['Rented Bike Count']]
    # Remove 'Rented Bike Count' from features
    X = X_original.drop('Rented Bike Count', axis=1)
    # Add original target to features
    X = pd.concat([X, y_original], axis=1)

else:
    # If 'Rented Bike Count' is already the target, just confirm
    print("'Rented Bike Count' is already the target variable.")
    y = y_original
    X = X_original

In [ ]:
print("\nNew feature columns:")
print(X.columns.tolist())
print("\nNew target variable:")
print(y.columns.tolist())

In [ ]:
# Check for missing values
print("\nMissing values in features:")
print(X.isnull().sum())
print("\nMissing values in target:")
print(y.isnull().sum())

In [ ]:
# Create a copy of the dataset to investigate periodic data
X_viz = X.copy()
y_viz = y.copy()

seasonal_data = X_viz.copy()
seasonal_data['Rented_Bike_Count'] = y_viz['Rented Bike Count']

In [ ]:
# Seasonal Bike Demand Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Seasonal Bike Sharing Demand Analysis', fontsize=16, fontweight='bold')

# Prepare data
seasonal_data = X_viz.copy()
seasonal_data['Rented_Bike_Count'] = y_viz['Rented Bike Count']

# Define season order
season_order = ['Winter', 'Spring', 'Summer', 'Autumn']
season_colors = ['#3498db', '#2ecc71', '#f39c12', '#e74c3c']  # Blue, Green, Orange, Red

# 1. Average Daily Demand by Season
seasonal_avg = seasonal_data.groupby('Seasons')['Rented_Bike_Count'].mean().reindex(season_order)

bars = ax1.bar(seasonal_avg.index, seasonal_avg.values, 
               color=season_colors, alpha=0.8, edgecolor='#333333', linewidth=1.2)
ax1.set_xlabel('Season', fontweight='bold')
ax1.set_ylabel('Average Daily Bike Rentals', fontweight='bold')
ax1.set_title('Average Daily Demand by Season', fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
             f'{int(height)}', ha='center', va='bottom', fontweight='bold')

# 2. Box plot showing distribution of demand by season
seasonal_data_ordered = []
season_labels = []
for season in season_order:
    season_data = seasonal_data[seasonal_data['Seasons'] == season]['Rented_Bike_Count']
    if len(season_data) > 0:
        seasonal_data_ordered.append(season_data)
        season_labels.append(season)

box_plot = ax2.boxplot(seasonal_data_ordered, tick_labels=season_labels, patch_artist=True)
for patch, color in zip(box_plot['boxes'], season_colors[:len(season_labels)]):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax2.set_xlabel('Season', fontweight='bold')
ax2.set_ylabel('Daily Bike Rentals', fontweight='bold')
ax2.set_title('Distribution of Daily Demand by Season', fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print seasonal summary statistics
print("Seasonal Bike Sharing Analysis")

for season in season_order:
    season_data = seasonal_data[seasonal_data['Seasons'] == season]['Rented_Bike_Count']
    if len(season_data) > 0:
        print(f"\n{season}:")
        print(f"  Average daily rentals: {season_data.mean():.1f}")
        print(f"  Peak daily demand: {season_data.max():,}")
        print(f"  Lowest daily demand: {season_data.min():,}")
        print(f"  Standard deviation: {season_data.std():.1f}")
        print(f"  Days of data: {len(season_data):,}")

# Compare seasons
print(f"\nSeasonal Comparisons:")

# Find best and worst seasons
season_averages = {}
for season in season_order:
    season_data = seasonal_data[seasonal_data['Seasons'] == season]['Rented_Bike_Count']
    if len(season_data) > 0:
        season_averages[season] = season_data.mean()

if season_averages:
    best_season = max(season_averages, key=season_averages.get)
    worst_season = min(season_averages, key=season_averages.get)
    
    print(f"Highest demand season: {best_season} ({season_averages[best_season]:.1f} avg daily)")
    print(f"Lowest demand season: {worst_season} ({season_averages[worst_season]:.1f} avg daily)")
    
    # Calculate difference
    seasonal_range = season_averages[best_season] - season_averages[worst_season]
    seasonal_range_pct = (seasonal_range / season_averages[worst_season]) * 100
    
    print(f"Seasonal variation: {seasonal_range:.1f} rentals ({seasonal_range_pct:.1f}% increase from lowest to highest)")

In [ ]:
# Daily Bike Demand Over the Year
fig, ax = plt.subplots(figsize=(15, 6))

# Prepare data
daily_data = X_viz.copy()
daily_data['Rented_Bike_Count'] = y_viz['Rented Bike Count']

# Convert Date to datetime if needed
if daily_data['Date'].dtype == 'object':
    daily_data['Date'] = pd.to_datetime(daily_data['Date'], format='%d/%m/%Y')

# Group by date and sum hourly data to get daily totals
daily_totals = daily_data.groupby('Date')['Rented_Bike_Count'].sum().reset_index()

# Plot daily demand
ax.plot(daily_totals['Date'], daily_totals['Rented_Bike_Count'], 
        color='#1f77b4', linewidth=1.5, alpha=0.8)

# Add a 7-day rolling average to smooth the trend
if len(daily_totals) > 7:
    rolling_avg = daily_totals['Rented_Bike_Count'].rolling(window=7, center=True).mean()
    ax.plot(daily_totals['Date'], rolling_avg, 
            color='#ff7f0e', linewidth=2.5, label='7-Day Moving Average')
    ax.legend()

ax.set_xlabel('Date', fontweight='bold', fontsize=12)
ax.set_ylabel('Daily Bike Rentals', fontweight='bold', fontsize=12)
ax.set_title('Daily Bike Demand Over the Year', fontweight='bold', fontsize=16)
ax.grid(True, alpha=0.3)

# Format x-axis to show months nicely
ax.tick_params(axis='x', rotation=45)
plt.tight_layout()
plt.show()

## Cyclical Encoding
For ML processes we often want to encode data for more efficient processing, however in this case, we need to choose an encoding which preserves the periodic patterns that will help us make our demand predictions. We can use a technique called Cyclical Encoding for the features in the dataset which are periodic in nature.  This will better help us capture seasonal, monthly, weekly and diurnal trends for our demand prediction.

The technique is borrowed from harmonic analysis and signal processing, where we place our periodic data on a unite circle rather than on a linear scale.  This helps us preserve the natural relationships, for example, between Sunday and Monday, Decemeber and Janaury, which are now neighbors rather than distance points.

This also helps smooth artificial jumps between time periods, for example between the first day of January and the last day of December, or between the end of Spring and the start of Summer.  The elegance here is that Euclidean distance in sin/cos space matches circular distance.

In [ ]:
# Add cyclical encoding for months and plot the difference
fig, axes = plt.subplots(1, 3, figsize=(18, 6))  # Changed from (2, 3) to (1, 3)
fig.suptitle('Cyclical Encoding Example: From Linear to Circular Representation', fontsize=16, fontweight='bold')

# Months of the year
months = np.arange(1, 13)  # Jan=1 to Dec=12
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Linear representation
axes[0].scatter(months, [0]*12, c=range(12), cmap='viridis', s=100)  # Changed from axes[0, 0] to axes[0]
for i, name in enumerate(month_names):
    axes[0].annotate(name, (months[i], 0), xytext=(0, 20), 
                       textcoords='offset points', ha='center')
axes[0].set_xlabel('Month (Linear)', fontweight='bold')
axes[0].set_title('Jan and Dec plotted linearly', fontweight='bold')
axes[0].set_ylim(-0.5, 1)
axes[0].grid(True, alpha=0.3)

# Cyclical representation using sin/cos
month_angles = 2 * np.pi * months / 12
month_sin = np.sin(month_angles)
month_cos = np.cos(month_angles)

# Plot on unit circle
circle = plt.Circle((0, 0), 1, fill=False, color='gray', linestyle='--', alpha=0.5)
axes[1].add_patch(circle)  # Changed from axes[0, 1] to axes[1]
axes[1].scatter(month_cos, month_sin, c=range(12), cmap='viridis', s=100)
for i, name in enumerate(month_names):
    axes[1].annotate(name, (month_cos[i], month_sin[i]), xytext=(5, 5), 
                       textcoords='offset points', fontsize=8)
axes[1].set_xlabel('Month Cosine', fontweight='bold')
axes[1].set_ylabel('Month Sine', fontweight='bold')
axes[1].set_title('Jan and Dec are neighbors on the unit circle', fontweight='bold')
axes[1].set_aspect('equal')
axes[1].grid(True, alpha=0.3)

# Encoding formulas
axes[2].text(0.1, 0.8, 'Cyclical Encoding Formulas:', fontsize=14, fontweight='bold', transform=axes[2].transAxes)  # Changed from axes[0, 2] to axes[2]
axes[2].text(0.1, 0.7, 'For month M (1-12):', fontsize=12, transform=axes[2].transAxes)
axes[2].text(0.1, 0.6, 'angle = 2π × M / 12', fontsize=12, transform=axes[2].transAxes, family='monospace')
axes[2].text(0.1, 0.5, 'month_sin = sin(angle)', fontsize=12, transform=axes[2].transAxes, family='monospace')
axes[2].text(0.1, 0.4, 'month_cos = cos(angle)', fontsize=12, transform=axes[2].transAxes, family='monospace')
axes[2].text(0.1, 0.25, 'Benefits:', fontsize=12, fontweight='bold', transform=axes[2].transAxes)
axes[2].text(0.1, 0.15, '• Preserves cyclical relationships', fontsize=10, transform=axes[2].transAxes)
axes[2].text(0.1, 0.1, '• Smooth transitions', fontsize=10, transform=axes[2].transAxes)
axes[2].text(0.1, 0.05, '• No arbitrary ordering', fontsize=10, transform=axes[2].transAxes)
axes[2].set_xlim(0, 1)
axes[2].set_ylim(0, 1)
axes[2].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
# Function to convert periodic data to cyclical encoding
def create_cyclical_encoding(X_data):
    X_cyclical = X_data.copy()
    
    X_cyclical['Date'] = pd.to_datetime(X_cyclical['Date'])
    
    X_cyclical['Month_sin'] = np.sin(2 * np.pi * X_cyclical['Month'] / 12)
    X_cyclical['Month_cos'] = np.cos(2 * np.pi * X_cyclical['Month'] / 12)
    
    X_cyclical['DayOfYear'] = X_cyclical['Date'].dt.dayofyear
    X_cyclical['DayOfYear_sin'] = np.sin(2 * np.pi * X_cyclical['DayOfYear'] / 365)
    X_cyclical['DayOfYear_cos'] = np.cos(2 * np.pi * X_cyclical['DayOfYear'] / 365)
    
    X_cyclical['Hour_sin'] = np.sin(2 * np.pi * X_cyclical['Hour'] / 24)
    X_cyclical['Hour_cos'] = np.cos(2 * np.pi * X_cyclical['Hour'] / 24)
    
    X_cyclical['DayOfWeek_sin'] = np.sin(2 * np.pi * X_cyclical['DayOfWeek'] / 7)
    X_cyclical['DayOfWeek_cos'] = np.cos(2 * np.pi * X_cyclical['DayOfWeek'] / 7)
    
    # Map seasons to numbers (0-3) in seasonal order
    season_mapping = {
        'Winter': 0,    # Start of cycle
        'Spring': 1,    # 1/4 through cycle  
        'Summer': 2,    # 1/2 through cycle
        'Autumn': 3     # 3/4 through cycle
    }
    
    # Create numeric season column
    X_cyclical['Season_numeric'] = X_cyclical['Seasons'].map(season_mapping)
    
    # Create seasonal cyclical encoding
    X_cyclical['Season_sin'] = np.sin(2 * np.pi * X_cyclical['Season_numeric'] / 4)
    X_cyclical['Season_cos'] = np.cos(2 * np.pi * X_cyclical['Season_numeric'] / 4)    
    
    return X_cyclical

In [ ]:
# Quick reset to avoid returning to the start of the notebook
seoul_bike_sharing_demand = fetch_ucirepo(id=560)
X_original = seoul_bike_sharing_demand.data.features
y_original = seoul_bike_sharing_demand.data.targets

# Make 'Rented Bike Count' the new target if it exists

if 'Rented Bike Count' in X_original.columns:
    # Make 'Rented Bike Count' the new target
    y = X_original[['Rented Bike Count']]
    # Remove 'Rented Bike Count' from features
    X = X_original.drop('Rented Bike Count', axis=1)
    # Add original target to features
    X = pd.concat([X, y_original], axis=1)

else:
    # If 'Rented Bike Count' is already the target, just confirm
    print("'Rented Bike Count' is already the target variable.")
    y = y_original
    X = X_original

In [ ]:
# 1. Convert date column to datetime and extract useful components. date format is DD/MM/YYYY
if 'Date' in X.columns:
    # Specify the correct date format as DD/MM/YYYY
    X['Date'] = pd.to_datetime(X['Date'], format='%d/%m/%Y')
    X['Year'] = X['Date'].dt.year
    X['Month'] = X['Date'].dt.month
    X['Day'] = X['Date'].dt.day
    X['DayOfWeek'] = X['Date'].dt.dayofweek


In [ ]:
# 2. Create cyclical encoding before adding one-encoding
X = create_cyclical_encoding(X)

# Drop Date after cyclical encoding
X = X.drop('Date', axis=1)

In [ ]:
# 3. Convert categorical features to numeric using one-hot encoding
X = pd.get_dummies(X, drop_first=True)

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_df = pd.DataFrame(X_train, columns=X_train.columns)

correlation_matrix = X_train_df.corr()

plt.figure(figsize=(12, 8))
# Use the calculated correlation matrix in the heatmap
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt='.2f')
plt.title("Feature Correlation Heatmap")
plt.show()

In [ ]:
# Manual removal of collinear features, since cyclical features will have intentionally high VIF
definitely_remove = ["DayOfYear", "Month", "Day", "Dew point temperature"]  
X_train_filtered = X_train.drop(columns=[f for f in definitely_remove if f in X_train.columns])
X_test_filtered = X_test.drop(columns=[f for f in definitely_remove if f in X_test.columns])

In [ ]:
# Standardize the filtered features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_filtered)
X_test_scaled = scaler.transform(X_test_filtered)

# **Model Development**
We break development into the following steps:
1. Create a baseline XGBoost model using default parameters
2. Use hyperparameter tuning with regularization to optimize the model
3. Implement early stopping to prevent overfitting
4. Evaluate the results and create visualizations

### Quick Reset
Before proceeding, let's create a quick way to start with fresh preprocessed data defined by our earlier scheme.

In [ ]:
def preprocess_data():
    """
    Minimal preprocessing pipeline for bike sharing demand data
    """
    # Load data
    seoul_bike_sharing_demand = fetch_ucirepo(id=560)
    X_original = seoul_bike_sharing_demand.data.features
    y_original = seoul_bike_sharing_demand.data.targets
    
    # Handle target variable
    if 'Rented Bike Count' in X_original.columns:
        y = X_original[['Rented Bike Count']]
        X = X_original.drop('Rented Bike Count', axis=1)
        X = pd.concat([X, y_original], axis=1)
    else:
        y = y_original
        X = X_original
        
    # Process dates and create cyclical features
    if 'Date' in X.columns:
        X['Date'] = pd.to_datetime(X['Date'], format='%d/%m/%Y')
        X['Year'] = X['Date'].dt.year
        X['Month'] = X['Date'].dt.month
        X['Day'] = X['Date'].dt.day
        X['DayOfWeek'] = X['Date'].dt.dayofweek
        
    # Create cyclical encoding and clean up
    X = create_cyclical_encoding(X)
    X = X.drop('Date', axis=1)
    X = pd.get_dummies(X, drop_first=True)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Remove collinear features
    definitely_remove = ["DayOfYear", "Month", "Day", "Dew point temperature"]
    actually_removed = [f for f in definitely_remove if f in X_train.columns]
    
    if actually_removed:
        X_train = X_train.drop(columns=actually_removed)
        X_test = X_test.drop(columns=actually_removed)
    
    # Standardize
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    return X_train_scaled, X_test_scaled, y_train, y_test, scaler, X_train.columns.tolist()

# Run preprocessing
X_train_scaled, X_test_scaled, y_train, y_test, scaler, feature_names = preprocess_data()
print(f"Data preprocessed: {len(feature_names)} features, {len(X_train_scaled)} training samples")

## 1. Create a Baseline Model and Check for Overfitting

In [ ]:
# Baseline XGBoost model with minimal configuration
baseline_model = xgb.XGBRegressor(
    random_state=42,
    n_jobs=-1,
    verbosity=0
)

# Train the baseline model
baseline_model.fit(X_train_scaled, y_train.values.ravel())

# Make predictions
baseline_train_preds = baseline_model.predict(X_train_scaled)
baseline_test_preds = baseline_model.predict(X_test_scaled)

# Calculate metrics
baseline_metrics = {
    'train_rmse': math.sqrt(mean_squared_error(y_train, baseline_train_preds)),
    'train_r2': r2_score(y_train, baseline_train_preds),
    'test_rmse': math.sqrt(mean_squared_error(y_test, baseline_test_preds)),
    'test_r2': r2_score(y_test, baseline_test_preds)
}

print(f"Training RMSE:  {baseline_metrics['train_rmse']:.2f}")
print(f"Training R²:    {baseline_metrics['train_r2']:.4f}")
print(f"Test RMSE:      {baseline_metrics['test_rmse']:.2f}")
print(f"Test R²:        {baseline_metrics['test_r2']:.4f}")

# Check for overfitting in baseline
rmse_gap = baseline_metrics['test_rmse'] - baseline_metrics['train_rmse']
print(f"\nOverfitting Check:")
print(f"RMSE Gap (Test - Train): {rmse_gap:.2f}")
if rmse_gap < 30:
    print("Minimal overfitting detected")
elif rmse_gap < 60:
    print("Moderate overfitting detected")
else:
    print("Significant overfitting detected")



We see that the baseline model overfits our training data with a RMSE Gap more than 60, we leave that for now and address overfitting for both the baseline and tuned models in step three.  

## 2. Perform Hyperparameter Tuning
Here we set up and conduct hyperparameter tuning to identify the optimal values for predicting bike usage. 

A table of the key XGBoost hyperparameters that we will use to control different aspects of model training, is provided:
| Parameter | Function | Impact |
|:-----------|:----------|:--------|
| `n_estimators` | Number of decision trees (rounds) | More trees = better fit |
| `learning_rate` | Step size for each tree's update | Lower = slower, more stable |
| `max_depth` | Maximum depth of each tree | Higher = more complex trees |
| `min_child_weight` | Minimum samples in leaf nodes | Higher = prevents overfitting |
| `reg_alpha` | L1 regularization (Lasso) | Higher = simpler model |
| `reg_lambda` | L2 regularization (Ridge) | Higher = smoother predictions |
| `subsample` | Fraction of samples per tree | Lower = more randomization |
| `colsample_bytree` | Fraction of features per tree | Lower = reduces correlation |

We can use gridsearch to evaluate different combinations of these parameters when we build our XGBoost decision trees.

In [ ]:
# Define comprehensive parameter grid for tuned model with regularization focus
param_grid_comprehensive = {
    # Number of boosting rounds
    'n_estimators': [100, 200, 300],
    
    # Learning rate - conservative values to prevent overfitting
    'learning_rate': [0.01, 0.05, 0.1],
    
    # Tree structure - moderate depth to capture patterns without overfitting
    'max_depth': [3, 4, 6],
    
    # Minimum child weight - higher values prevent overfitting
    'min_child_weight': [1, 3, 5],
    
    # L1 regularization - encourages sparsity
    'reg_alpha': [0, 0.1, 0.5],
    
    # L2 regularization - smooths predictions
    'reg_lambda': [1, 1.5, 2],
    
    # Subsampling - reduces overfitting through randomization
    'subsample': [0.7, 0.8, 0.9],
    
    # Feature subsampling - reduces correlation between trees
    'colsample_bytree': [0.7, 0.8, 0.9]
}

## 3. Early Stopping with Cross Validation

### First we address overfitting with the baseline model

We can adjust our baseline model by adding hyperparameters, first we try adjusting the learning rate and tree depth:
| Parameter | Function | Impact |
|:-----------|:----------|:--------|
| `n_estimators` | Number of decision trees (rounds) | More trees = better fit |
| `learning_rate` | Step size for each tree's update | Lower = slower, more stable |
| `max_depth` | Maximum depth of each tree | Higher = more complex trees |

In [ ]:
# Simple baseline with basic settings
baseline_model = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    n_jobs=-1
)

# Train and evaluate
baseline_model.fit(X_train_scaled, y_train.values.ravel())
baseline_train_preds = baseline_model.predict(X_train_scaled)
baseline_test_preds = baseline_model.predict(X_test_scaled)

# Calculate metrics
baseline_train_rmse = math.sqrt(mean_squared_error(y_train, baseline_train_preds))
baseline_test_rmse = math.sqrt(mean_squared_error(y_test, baseline_test_preds))
baseline_train_r2 = r2_score(y_train, baseline_train_preds)
baseline_test_r2 = r2_score(y_test, baseline_test_preds)

print(f"BASELINE PERFORMANCE:")
print(f"Train RMSE: {baseline_train_rmse:.2f} | Test RMSE: {baseline_test_rmse:.2f}")
print(f"Train R²:   {baseline_train_r2:.4f} | Test R²:   {baseline_test_r2:.4f}")
print(f"Overfitting Gap: {baseline_test_rmse:.2f} -  {baseline_train_rmse:.2f} = {baseline_test_rmse - baseline_train_rmse:.2f}")

This improved our overfitting reducing the gap from 91.59 to 55.37 in this new model with minimal regularization.  This model has moderate overfitting but is a huge improvement.  Still, we can do better by adding stronger regularization.
| Parameter | Function | Impact |
|:-----------|:----------|:--------|
| `n_estimators` | Number of decision trees (rounds) | More trees = better fit |
| `learning_rate` | Step size for each tree's update | Lower = slower, more stable |
| `max_depth` | Maximum depth of each tree | Higher = more complex trees |
| `min_child_weight` | Minimum samples in leaf nodes | Higher = prevents overfitting |
| `reg_alpha` | L1 regularization (Lasso) | Higher = simpler model |
| `reg_lambda` | L2 regularization (Ridge) | Higher = smoother predictions |
| `subsample` | Fraction of samples per tree | Lower = more randomization |
| `colsample_bytree` | Fraction of features per tree | Lower = reduces correlation |

In [ ]:
# Enhanced baseline with regularization to address overfitting
baseline_regularized = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.08,          # Slightly lower
    max_depth=5,                 # Reduced depth
    min_child_weight=3,          # Higher minimum
    reg_alpha=0.5,               # L1 regularization
    reg_lambda=2,                # L2 regularization
    subsample=0.8,               # Sample 80% of data
    colsample_bytree=0.8,        # Sample 80% of features
    random_state=42,
    n_jobs=-1
)

# Train and evaluate
baseline_regularized.fit(X_train_scaled, y_train.values.ravel())
reg_train_preds = baseline_regularized.predict(X_train_scaled)
reg_test_preds = baseline_regularized.predict(X_test_scaled)

# Calculate metrics
reg_train_rmse = math.sqrt(mean_squared_error(y_train, reg_train_preds))
reg_test_rmse = math.sqrt(mean_squared_error(y_test, reg_test_preds))
reg_train_r2 = r2_score(y_train, reg_train_preds)
reg_test_r2 = r2_score(y_test, reg_test_preds)

print(f"REGULARIZED BASELINE PERFORMANCE:")
print(f"Train RMSE: {reg_train_rmse:.2f} | Test RMSE: {reg_test_rmse:.2f}")
print(f"Train R²:   {reg_train_r2:.4f} | Test R²:   {reg_test_r2:.4f}")
print(f"Overfitting Gap: {reg_test_rmse - reg_train_rmse:.2f}")

print(f"\nBASELINE IMPROVEMENT:")
print(f"Gap Reduction: {(baseline_test_rmse - baseline_train_rmse) - (reg_test_rmse - reg_train_rmse):.2f}")


We'll use this new baseline model as it shows a significant improvement in generalization (34.38 is a much lower overfitting gap). . 

### Building a seasonally-aware model while staying vigiliant of overfitting
To capture the complex seasonal interactions, we now turn to Grid Search to try to create a more seasonally-aware model. Our max depth parameter helps us capture complex seasonal interactions, reg_alpha and reg_lambda balance feature importance so cyclical features aren't overshadowed.  We adjust our learning rate so we can learn seasonal patterns slowly, and employ early stopping to ensure that the seasonally-aware model does not overfit.

In [ ]:
# Anti-overfitting parameter grid specifically designed for seasonal patterns
param_grid_seasonal = {
    # Conservative number of trees
    'n_estimators': [100, 150, 200],
    
    # Lower learning rates for stable seasonal learning
    'learning_rate': [0.01, 0.05, 0.08],
    
    # Moderate tree depths to capture seasonal interactions without overfitting
    'max_depth': [4, 5, 6],
    
    # Higher minimum child weight for generalization
    'min_child_weight': [3, 5, 7],
    
    # Strong L1 regularization for feature selection (helps cyclical features shine)
    'reg_alpha': [0.5, 1.0, 2.0],
    
    # Strong L2 regularization for smooth predictions
    'reg_lambda': [2, 3, 4],
    
    # Aggressive subsampling for randomization (prevents seasonal overfitting)
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8]
}

print(f"Parameter combinations: {np.prod([len(v) for v in param_grid_seasonal.values()])}")
print("Focus: Optimize cyclical feature usage while preventing overfitting")

# Time Series Cross-Validation (preserves temporal relationships)
tscv = TimeSeriesSplit(n_splits=5, test_size=len(X_train_scaled)//8)
print("Using TimeSeriesSplit to respect temporal order in bike demand data")

# XGBoost configured for seasonal data
xgb_seasonal = xgb.XGBRegressor(
    random_state=42,
    n_jobs=-1,
    verbosity=0,
    objective='reg:squarederror',
    eval_metric='rmse'
)

# Grid Search with seasonal focus
grid_search_seasonal = GridSearchCV(
    estimator=xgb_seasonal,
    param_grid=param_grid_seasonal,
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=7,   # number of cores to use (adjust for your system)
    verbose=1,  # 0-suppress progress, 1-show minimal progress, 2-show progress with timing, 3- detailed progress
    return_train_score=True
)

print("Starting seasonal-aware grid search...")
print("This will find optimal parameters for cyclical feature utilization, and will take some time to finish.")
print("Adjusting n_jobs can speed up the process based on the number of cores available.  Verbose progress is turned off, but feel free to adjust.")
# Fit the grid search
grid_search_seasonal.fit(X_train_scaled, y_train.values.ravel())

print("Seasonal grid search completed!")

# Extract best results
best_params_seasonal = grid_search_seasonal.best_params_
best_cv_score = grid_search_seasonal.best_score_

print(f"\nBEST SEASONAL PARAMETERS:")
print("-" * 35)
for param, value in best_params_seasonal.items():
    print(f"{param:18s}: {value}")

print(f"\nBest CV Score (Neg MSE): {best_cv_score:.2f}")
print(f"Best CV RMSE: {math.sqrt(-best_cv_score):.2f}")

# ================================================================================================
# FINAL MODEL WITH EARLY STOPPING
# ================================================================================================

print(f"\nTRAINING FINAL SEASONAL MODEL WITH EARLY STOPPING")
print("-" * 50)

# Create validation split for early stopping
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    X_train_scaled, y_train.values.ravel(), test_size=0.15, random_state=42
)

print(f"Training split: {X_train_final.shape}")
print(f"Validation split: {X_val_final.shape}")

# Final seasonal model with best parameters + early stopping
final_model_balanced = xgb.XGBRegressor(
    n_estimators=280,           # More trees = higher overfitting risk (was 320, tried 250, 200, 150)
    learning_rate=0.045,        # Moderate (was 0.05, tried 0.08, 0.03)
    max_depth=6,                # Controls amount of complexity for cyclical features (was 5, tried 4)  
    min_child_weight=3,         # Controls amount of complexity for cyclical features (was 2.5, tried 7)
    reg_alpha=1.0,              # Controls overfitting (was 0.9, tried 0.7, 0.5, 2.5)  
    reg_lambda=3.2,             # Controls overfitting (was 3.2, tried 2.0, 4.0)
    subsample=0.82,             # Less aggressive (was 0.85, tried 0.8, 0.7, 0.6)
    colsample_bytree=0.8,       # Helps cyclical features (was 0.85, tried 0.7, 0.6)
    early_stopping_rounds=18,   # More patient (was 20, tried 25, 15, 8)
    random_state=42,
    n_jobs=-1
)

# Train with early stopping validation
print("Training seasonal model with early stopping...")
final_model_balanced.fit(
    X_train_final, y_train_final,
    eval_set=[(X_val_final, y_val_final)],
    verbose=False
)

print("Seasonal model training completed!")

# Check early stopping results
if hasattr(final_model_balanced, 'best_iteration') and final_model_balanced.best_iteration:
    print(f"Early stopping triggered at iteration: {final_model_balanced.best_iteration}")
    trees_saved = 280 - final_model_balanced.best_iteration
    print(f"Trees saved from overfitting: {trees_saved}")
else:
    print(f"Used all 280 trees (early stopping didn't trigger)")

# Evaluate final model - FIX THESE LINES:
balanced_train_preds = final_model_balanced.predict(X_train_scaled)
balanced_test_preds = final_model_balanced.predict(X_test_scaled)

balanced_train_rmse = math.sqrt(mean_squared_error(y_train, balanced_train_preds))
balanced_test_rmse = math.sqrt(mean_squared_error(y_test, balanced_test_preds))
balanced_train_r2 = r2_score(y_train, balanced_train_preds)
balanced_test_r2 = r2_score(y_test, balanced_test_preds)

print(f"\nFINAL BALANCED MODEL PERFORMANCE:")
print("-" * 40)
print(f"Training RMSE:  {balanced_train_rmse:.2f}")
print(f"Training R²:    {balanced_train_r2:.4f}")
print(f"Test RMSE:      {balanced_test_rmse:.2f}")
print(f"Test R²:        {balanced_test_r2:.4f}")

# Overfitting analysis
balanced_gap = balanced_test_rmse - balanced_train_rmse
print(f"\nOverfitting Gap: {balanced_gap:.2f}")

if balanced_gap < 30:
    print("Excellent generalization achieved!")
elif balanced_gap < 50:
    print("Good generalization achieved!")
else:
    print("Some overfitting remains - but model still usable")

# Feature importance analysis
print(f"\nCYCLICAL FEATURE IMPORTANCE:")
print("-" * 32)

feature_importance = pd.DataFrame({
    'feature': feature_names,
    'importance': final_model_balanced.feature_importances_
}).sort_values('importance', ascending=False)

# Highlight cyclical features
cyclical_features = feature_importance[feature_importance['feature'].str.contains('_sin|_cos')].head(8)

if len(cyclical_features) > 0:
    print(f"Top cyclical features in final model:")
    for i, (_, row) in enumerate(cyclical_features.iterrows(), 1):
        print(f"  {i}. {row['feature']:<20s}: {row['importance']:.4f}")
        
    cyclical_importance_sum = cyclical_features['importance'].sum()
    total_importance = feature_importance['importance'].sum()
    cyclical_percentage = (cyclical_importance_sum / total_importance) * 100
    
    print(f"\nCyclical features account for {cyclical_percentage:.1f}% of total importance")
    print("Seasonal patterns successfully captured!")
else:
    print("No cyclical features found in top importance - check feature engineering")

print(f"\nBALANCED MODEL READY FOR DEPLOYMENT!")
print(f"The model successfully leverages cyclical encodings for bike demand prediction")

## 4. Evaluation and Visualizations

In [ ]:
# Prepare combined data
y_actual_all = np.concatenate([y_train.values.ravel(), y_test.values.ravel()])
y_pred_all = np.concatenate([balanced_train_preds, balanced_test_preds])

# Create date range (adjust start_date to match your data period)
start_date = pd.Timestamp('2016-12-01')  # Adjust based on your actual data period
dates = pd.date_range(start=start_date, periods=len(y_actual_all), freq='h')

# Create DataFrame
df_combined = pd.DataFrame({
    'Date': dates,
    'Actual': y_actual_all,
    'Predicted': y_pred_all
})

# Aggregate to weekly
df_weekly = df_combined.set_index('Date').resample('W').mean().reset_index()

# Create the time series plot
fig, ax = plt.subplots(figsize=(16, 8))

# Plot actual and predicted lines
ax.plot(df_weekly['Date'], df_weekly['Actual'], 
        color='#1f77b4', linewidth=2, alpha=0.8, label='Actual')
ax.plot(df_weekly['Date'], df_weekly['Predicted'], 
        color='#ff4444', linewidth=2, alpha=0.8, label='Predictions')

# Add train/test split line
# Calculate split date based on train/test sizes
train_size = len(y_train)
total_size = len(y_actual_all)
split_ratio = train_size / total_size

# Find corresponding date in weekly data
split_week_idx = int(len(df_weekly) * split_ratio)
if split_week_idx < len(df_weekly):
    split_date = df_weekly.iloc[split_week_idx]['Date']
    ax.axvline(x=split_date, color='green', linestyle='--', alpha=0.7, linewidth=2,
              label=f'Train/Test Split ({split_date.strftime("%Y-%m-%d")})')

# Formatting
ax.set_xlabel('Date', fontweight='bold', fontsize=12)
ax.set_ylabel('Weekly Average Bikes Shared', fontweight='bold', fontsize=12)
ax.set_title('Weekly Bike Demand: Actual vs Predicted Over Time', fontweight='bold', fontsize=14)

# Legend and grid
ax.legend(loc='upper right', frameon=True, fancybox=True, shadow=True, fontsize=12)
ax.grid(True, alpha=0.3)

# Format x-axis to show months
from matplotlib.dates import DateFormatter, MonthLocator
ax.xaxis.set_major_locator(MonthLocator())
ax.xaxis.set_major_formatter(DateFormatter('%Y-%m'))
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"Weekly Time Series Statistics:")
print(f"Total weeks: {len(df_weekly)}")
print(f"Date range: {df_weekly['Date'].min().strftime('%Y-%m-%d')} to {df_weekly['Date'].max().strftime('%Y-%m-%d')}")
print(f"Weekly RMSE: {np.sqrt(((df_weekly['Actual'] - df_weekly['Predicted'])**2).mean()):.2f}")


In [ ]:
def analyze_seasonal_patterns(df_weekly):
    """
    Analyze how well the model captures seasonal patterns
    """   
    # Add seasonal information
    df_analysis = df_weekly.copy()
    df_analysis['Month'] = df_analysis['Date'].dt.month
    df_analysis['Season'] = df_analysis['Month'].map({
        12: 'Winter', 1: 'Winter', 2: 'Winter',
        3: 'Spring', 4: 'Spring', 5: 'Spring',
        6: 'Summer', 7: 'Summer', 8: 'Summer',
        9: 'Autumn', 10: 'Autumn', 11: 'Autumn'
    })
    
    # Calculate seasonal averages
    seasonal_comparison = df_analysis.groupby('Season')[['Actual', 'Predicted']].mean()
    
    print("Seasonal Performance:")
    for season in ['Spring', 'Summer', 'Autumn', 'Winter']:
        if season in seasonal_comparison.index:
            actual = seasonal_comparison.loc[season, 'Actual']
            pred = seasonal_comparison.loc[season, 'Predicted']
            error = abs(actual - pred)
            error_pct = (error / actual) * 100
            print(f"  {season:6s}: Actual={actual:6.1f}, Predicted={pred:6.1f}, Error={error_pct:4.1f}%")
    
    # Create seasonal comparison visualization
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    fig.suptitle('Seasonal Performance Analysis', fontsize=16, fontweight='bold')
    
    # Bar chart comparison
    seasons = seasonal_comparison.index
    x = np.arange(len(seasons))
    width = 0.35
    
    bars1 = ax1.bar(x - width/2, seasonal_comparison['Actual'], width, 
                   label='Actual', color='#1f77b4', alpha=0.8)
    bars2 = ax1.bar(x + width/2, seasonal_comparison['Predicted'], width,
                   label='Predicted', color='#ff4444', alpha=0.8)
    
    ax1.set_xlabel('Season', fontweight='bold')
    ax1.set_ylabel('Average Weekly Bikes', fontweight='bold')
    ax1.set_title('Seasonal Averages: Actual vs Predicted', fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(seasons)
    ax1.legend()
    ax1.grid(True, alpha=0.3, axis='y')
    
    # Error by season
    seasonal_errors = [abs(seasonal_comparison.loc[season, 'Actual'] - seasonal_comparison.loc[season, 'Predicted']) 
                      for season in seasons]
    bars3 = ax2.bar(seasons, seasonal_errors, color=['#2ecc71', '#f39c12', '#e74c3c', '#3498db'], alpha=0.8)
    
    ax2.set_xlabel('Season', fontweight='bold')
    ax2.set_ylabel('Absolute Prediction Error', fontweight='bold')
    ax2.set_title('Prediction Error by Season', fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    return seasonal_comparison

# Analyze seasonal patterns
seasonal_analysis = analyze_seasonal_patterns(df_weekly)

In [ ]:
def create_monthly_bar_comparison(y_actual, y_predicted, start_date='2016-12-01'):
    """
    Create monthly grouped bar comparison
    """
    # Create DataFrame
    dates = pd.date_range(start=start_date, periods=len(y_actual), freq='h')
    df = pd.DataFrame({
        'Date': dates,
        'Actual': y_actual.ravel() if hasattr(y_actual, 'ravel') else y_actual,
        'Predicted': y_predicted.ravel() if hasattr(y_predicted, 'ravel') else y_predicted
    })
    
    # Aggregate to monthly
    df_monthly = df.set_index('Date').resample('ME').mean().reset_index()
    
    print(f"Monthly data points: {len(df_monthly)}")
    
    # Create the plot
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Bar positions
    x = np.arange(len(df_monthly))
    width = 0.35
    
    # Create bars
    bars1 = ax.bar(x - width/2, df_monthly['Actual'], width, 
                   label='Actual', color='#1f77b4', alpha=0.8, edgecolor='black', linewidth=0.5)
    bars2 = ax.bar(x + width/2, df_monthly['Predicted'], width,
                   label='Predicted', color='#ff4444', alpha=0.8, edgecolor='black', linewidth=0.5)
    
    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
               f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    for bar in bars2:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
               f'{int(height)}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # Formatting
    ax.set_xlabel('Month', fontweight='bold', fontsize=12)
    ax.set_ylabel('Average Bikes per Hour', fontweight='bold', fontsize=12)
    ax.set_title('Monthly Bike Demand: Actual vs Predicted Comparison', fontweight='bold', fontsize=14)
    
    # X-axis labels
    ax.set_xticks(x)
    ax.set_xticklabels([date.strftime('%Y-%m') for date in df_monthly['Date']], 
                       rotation=45, ha='right')
    
    ax.legend(fontsize=12)
    ax.grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Monthly statistics
    monthly_rmse = np.sqrt(((df_monthly['Actual'] - df_monthly['Predicted'])**2).mean())
    print(f"\nMonthly Statistics:")
    print(f"Monthly RMSE: {monthly_rmse:.2f}")
    print(f"Mean monthly actual: {df_monthly['Actual'].mean():.1f}")
    print(f"Mean monthly predicted: {df_monthly['Predicted'].mean():.1f}")
    
    # Find best and worst months
    df_monthly['Error'] = abs(df_monthly['Actual'] - df_monthly['Predicted'])
    best_month = df_monthly.loc[df_monthly['Error'].idxmin(), 'Date']
    worst_month = df_monthly.loc[df_monthly['Error'].idxmax(), 'Date']
    
    print(f"Best prediction month: {best_month.strftime('%Y-%m')} (Error: {df_monthly['Error'].min():.1f})")
    print(f"Worst prediction month: {worst_month.strftime('%Y-%m')} (Error: {df_monthly['Error'].max():.1f})")
    
    return df_monthly

# Create monthly bar comparison
monthly_bars = create_monthly_bar_comparison(y_actual_all, y_pred_all)

In [ ]:
# Baseline metrics (from regularized baseline)
baseline_train_rmse = math.sqrt(mean_squared_error(y_train, baseline_train_preds))  # Your baseline predictions
baseline_test_rmse = math.sqrt(mean_squared_error(y_test, baseline_test_preds))

# Final model metrics (from balanced final model)
final_train_rmse = math.sqrt(mean_squared_error(y_train, balanced_train_preds))
final_test_rmse = math.sqrt(mean_squared_error(y_test, balanced_test_preds))

# R² scores
baseline_train_r2 = r2_score(y_train, baseline_train_preds)
baseline_test_r2 = r2_score(y_test, baseline_test_preds)
final_train_r2 = r2_score(y_train, balanced_train_preds)
final_test_r2 = r2_score(y_test, balanced_test_preds)

# Overfitting gaps
baseline_gap = baseline_test_rmse - baseline_train_rmse
final_gap = final_test_rmse - final_train_rmse

# Create the comparison chart
plt.figure(figsize=(16, 6))
fig = plt.gcf()
fig.patch.set_facecolor('#f8f9fa')

# Define colors
colors = ['#39A0ED', '#FF5E5B']  # Blue for Baseline, Coral for Final

# Model names
models = ['Regularized\nBaseline', 'Final Tuned\nModel']

# RMSE comparison (subplot 1)
plt.subplot(1, 3, 1)
test_rmse_values = [baseline_test_rmse, final_test_rmse]
bars1 = plt.bar(models, test_rmse_values, color=colors,
               edgecolor='white', linewidth=0.8, width=0.7)

# Add values on top of bars
for bar in bars1:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + height*0.01,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.title('Test RMSE Comparison\n(lower is better)', fontsize=14, pad=20, fontweight='bold', color='#333333')
plt.ylabel('Test RMSE', fontsize=12, color='#333333')
plt.grid(axis='y', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add best performance line
min_rmse = min(test_rmse_values)
plt.axhline(y=min_rmse, color='#333333', linestyle='--', alpha=0.5)
plt.text(len(models)-1, min_rmse + min_rmse*0.05, f'Best: {min_rmse:.2f}',
         ha='right', va='bottom', color='#333333', alpha=0.7, fontweight='bold')

# R² comparison (subplot 2)
plt.subplot(1, 3, 2)
r2_values = [baseline_test_r2, final_test_r2]
bars2 = plt.bar(models, r2_values, color=colors,
               edgecolor='white', linewidth=0.8, width=0.7)

# Add values on top of bars
for bar in bars2:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.005,
            f'{height:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.title('Test R² Comparison\n(higher is better)', fontsize=14, pad=20, fontweight='bold', color='#333333')
plt.ylabel('Test R²', fontsize=12, color='#333333')
plt.grid(axis='y', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add best performance line
max_r2 = max(r2_values)
plt.axhline(y=max_r2, color='#333333', linestyle='--', alpha=0.5)
plt.text(len(models)-1, max_r2 - 0.01, f'Best: {max_r2:.4f}',
         ha='right', va='top', color='#333333', alpha=0.7, fontweight='bold')

# Overfitting Gap comparison (subplot 3)
plt.subplot(1, 3, 3)
gap_values = [baseline_gap, final_gap]
bars3 = plt.bar(models, gap_values, color=colors,
               edgecolor='white', linewidth=0.8, width=0.7)

# Add values on top of bars
for bar in bars3:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + height*0.02,
            f'{height:.2f}', ha='center', va='bottom', fontweight='bold', fontsize=11)

plt.title('Overfitting Gap\n(lower is better)', fontsize=14, pad=20, fontweight='bold', color='#333333')
plt.ylabel('Gap (Test - Train RMSE)', fontsize=12, color='#333333')
plt.grid(axis='y', alpha=0.3)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add best performance line
min_gap = min(gap_values)
plt.axhline(y=min_gap, color='#333333', linestyle='--', alpha=0.5)
plt.text(len(models)-1, min_gap + min_gap*0.1, f'Best: {min_gap:.2f}',
         ha='right', va='bottom', color='#333333', alpha=0.7, fontweight='bold')

plt.tight_layout()
plt.show()